In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("../../combined_file.csv")
df = df[df["common_name"] != "ok to plant vacant"]

C:\Users\Laptop\AppData\Local\Temp\ipykernel_18748\3877153638.py:2: DtypeWarning: Columns (0,1,2,8,9,12,13,15,17,18,19,21,22,23,24,25,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../combined_file.csv")


In [2]:
df.columns


Index(['most_recent_observation', 'most_recent_observation_type',
       'common_name', 'scientific_name', 'city', 'state',
       'longitude_coordinate', 'latitude_coordinate', 'address', 'condition',
       'height_M', 'native', 'height_binned_M',
       'diameter_breast_height_binned_CM', 'greater_metro', 'city_ID',
       'tree_ID', 'planted_date', 'retired_date', 'location_type', 'zipcode',
       'neighborhood', 'location_name', 'ward', 'district', 'overhead_utility',
       'diameter_breast_height_CM', 'percent_population', 'column_name',
       'definition'],
      dtype='object')

In [7]:

top10_df = df.groupby(["common_name"])["common_name"].count().sort_values(ascending=False).head(10)

print("Total trees: " + str(top10_df.sum()))
top10_df

Total trees: 1042325


common_name
Norway maple         142560
London planetree     122493
Red maple            121825
Crape myrtle         121775
Green ash             98282
Callery pear          94271
Honeylocust           91954
Live oak              83648
Pin oak               83187
Littleleaf linden     82330
Name: common_name, dtype: int64

In [8]:
top10 = top10_df.index.to_list()
top10

['Norway maple',
 'London planetree',
 'Red maple',
 'Crape myrtle',
 'Green ash',
 'Callery pear',
 'Honeylocust',
 'Live oak',
 'Pin oak',
 'Littleleaf linden']

In [9]:
df1 = df.copy()
df1 = df1[["common_name", "latitude_coordinate", "longitude_coordinate"]]
df1.loc[~df1['common_name'].isin(top10), 'common_name'] = np.nan
df1 = df1.dropna(subset=['latitude_coordinate', 'longitude_coordinate'])

In [10]:
len(df1)

971888

In [14]:
df1.to_csv("../datasets/dd.csv", index=False)

In [15]:
import json
data = df1.copy()

features = []
for index, row in data.iterrows():
    latitude = row['latitude_coordinate']
    longitude = row['longitude_coordinate']
    name = row['common_name']

    feature = {
        'type': 'Feature',
        'properties': {'name': name},
        'geometry': {
            'type': 'Point',
            'coordinates': [float(longitude), float(latitude)]
        }
    }
    features.append(feature)

geojson = {
'type': 'FeatureCollection',
'features': features
}

with open('../datasets/dd.geojson', 'w') as f:
    json.dump(geojson, f)

""

''